# 하스켈의 이론적 토대 기초

하스켈 프로그래밍 언어의 이론적 토대는 **람다 대수**(Lambda calculus, $\lambda$-calculus)이다.
람다 대수를 간략하게 소개하고 함수형 프로그래밍과의 연관성을 설명한다. 

다루는 내용은 다음과 같다.

* 람다 대수
* 람다 계산
* 단순 유형 람다 대수

## 람다 대수

람다 대수는 '계산이란 무엇인가'와 '어떻게 계산하는가'에 대한 대답을 
수학적으로 엄밀하게 정의할 수 있는 
형식 체계(formal system)를 가리킨다.  
즉, 람다 대수 체계가 지원하는 언어를 이용하여 계산과 계산법을 엄밀하게 정의할 수 있다. 

람다 대수는 계산(computation)을 함수와 함수의 적용 단 두 개의 개념을 이용하여 정의하며
계산 방법은 베타-계산법(beta-reduction, $\beta$-reduction)으로 정의한다. 

### 언어

람다 대수 언어 $\Lambda$의 알파벳은 다음 두 부류이다.

* 무한이 많은 변수: $x, y, z, ...$ 등등
* 기호 두 개: '$\lambda$'(람다)와 '$.$'(점)

#### 람다항

일반 프로그래밍 언어에서 표현식(expression)으로 불리는 람다항(lambda term)을 다음과 같이 귀납적으로 정의할 수 있다.

* 변수항: 모든 변수 $x$는 람다항이다. 
* 추상화: $M$이 람다항이면 $\lambda x. M$도 람다항이다.
* 적용: $M$과 $N$ 모두 람다항이면 $M \, N$도 람다항이다.

**참조:** 람다항 표현을 명확하기 위해 괄호를 사용할 수 있다.
* $(M\, N)$
* $\lambda x. (M\, N)$
* $(\lambda x. M)\, N$
* $(M\, N)\, L$
* $M\, (N \, L)$

**왼쪽 우선 결합성:**(left-associativity) $(M_1\, M_2\, M_3\, \dots\, M_k)$는 다음을 가리킨다.

$$(\dots ((M_1\, M_2)\,M_3\, \dots\, M_k)$$

### 처치 인코딩(Church encoding)

람다 대수의 언어가 매우 단순하다. 
하지만 람다 대수와 튜링 기계는 이론적으로 동급이다.
이를 증명하기 위해 처치는 자연수와 연산을 모두 람다 대수에서 구현할 수 있음을 입증하기 위해
자연수와 관련 연산을 람다항으로 구현하였다.
이를 **처치 인코딩**이라 부른다. 

#### 처치 자연수(Church numerals)

자연수를 람다항으로 표현할 수 있다. 
아래 표에서 $f^{(n)}\, x := \underbrace{f\, (f\, (\cdots (f}_{n\text{ 번}} \, x)))$ 이다.

* 0 = $\lambda f. \lambda x. x$
* 1 = $\lambda f. \lambda x. f\, x$
* 2 = $\lambda f. \lambda x. f\, (f\, x)$
* 3 = $\lambda f. \lambda x. f\, (f\, (f\, x))$

    ...

* n = $\lambda f. \lambda x. f^{(n)}\, x$

#### 처치 자연수 연산 함수

자연수의 덧셈, 뺄셈, 곱셈 등을 모두 람다항으로 표현할 수 있다.

* 덧셈: plus = $\lambda m.\lambda n.\lambda f.\lambda x. m\, f\, (n\, f\, x)$
* +1: succ = $\lambda n.\lambda f.\lambda x. f\, (n\, f\, x)$
* 곱셈: mult = $\lambda m.\lambda n.\lambda f.\lambda x. m\, (n\, f)\, x$
* -1: pred = $\lambda n.\lambda f.\lambda x.n\, (\lambda g.\lambda h. h\, (g\, f))\,(\lambda u.x)\,(\lambda u.u)$
* 뺄셈: minus = $\lambda m.\lambda n. (n\, \text{pred})\, m$
* 지수승: exp = $\lambda m.\lambda n. n\, m$

위 람다항들을 이용하여 모든 자연수 연산을 모델링할 수 있다.
하지만 연산을 하려면 계산을 먼저 정의하며, 
이를 위해 **베타-계산법**($\lambda$-reduction)을 사용한다.

### 베타-계산법

베타-계산법은 함수를 적절한 인자와 함께 호출했을 때 벌어지는 현상을 묘사한다.
함수와 인자를 구분하는 방법을 먼저 알아야 한다.

* 함수: $\lambda x.M$ 모양의 람다항을 함수로 본다.
* 인자: $M \, N$ 모양의 람다항 중에서 $N$이 인자의 역할을 수행한다.

따라서 함수 호출은 다음 모양을 갖는 람다항으로 모델링 된다.

$$(\lambda x.M)\, N$$

이제 남아 있는 질문은 함수 호출 후 실제 계산을 어떻게 모델링 할 것인가이다. 
이를 위해 파이썬에서 람다 함수를 호출할 때 발생하는 계산 과정을 살펴본다.
파이썬에서 이름 없는 함수를 정의할 때 사용하는 람다 표현식이 바로 추상화된 람다항이다. 
또한 람다 표현식의 호출이 바로 위 모양과 사실상 동일하다. 
파이썬에서 람다 함수 호출을 할 때의 연산과정을 아래 예제가 잘 보여준다. 

```python
(lambda x: x^2 + 1)(2) = 2^2 + 1    (x를 2로 대체)
                       = 4 + 1      (지수함수 호출 및 계산)
                       = 5          (덧셈함수 호출 및 계산)
```

위 계산을 람다 대수에서 표현하면 다음과 같다. 

$$
(\lambda x. \text{plus}\, (\text{exp}\, x\, 2)\, 1)\, 2
= \text{plus}\, (\text{exp}\, 2\, 2)\, 1)
= \text{plus}\, 4\, 1 
= 5
$$

위 계산과정에서 사용된 규칙은 놀랍게도 베타-규칙($\beta$-rule) 단 한 개이며 다음과 같다. 

$$(\lambda x.M)\, N \to_\beta M[x:=N]\qquad\qquad (\beta\text{-rule})$$

단, $M[x:=N]$은 람다항 $M$에서 사용된 변수 $x$를 람다항 $N$으로 대체하는 것을 가리킨다.

#### 베타-계산

베타-계산은 간단히 말해 앞서 정의된 규칙 $\to_\beta$를 반복적으로 실행하는 것을 의미한다.
베타-계산에 대한 엄밀한 정의는 이 강의의 수준을 넘어서며, 여기서는 베타-계산에 대한 약간의 감을 전달하는 것에 만족한다. 

### 튜링 완전성(Turing completeness)

람다 대수는 튜링-완전하다. 
즉, 튜링 기계에서 구현되는 모든 함수를 람다 대수에서 구현할 수 있으며, 그 역도 성립한다.

이는 현재 우리가 사용하는 컴퓨터와 람다 대수가 이론적으로는 동등함을 의미한다. 
실제로 현대 컴퓨터는 폰 노이만 컴퓨터에 뿌리를 두고 있으며, 폰 노이만 컴퓨터는 튜링 기계와 이론적으로 동일하다.

튜링과 폰 노이만과이 현대적 컴퓨터 개발에 어떤 기여를 했고 서로 어떻게 경쟁했는지 잘 알려져 있으며,
한국리눅스문서화프로젝트(KLDP)의 블로그에 이와 관련된 재밌는 글이 있어서 추천한다. 

링크: [앨런 튜링과 폰 노이만](https://joone.net/2016/11/15/앨런-튜링과-폰노이만/#comment-240)

## 하스켈과 람다 대수

하스켈에서 정의되는 모든 함수와 데이터는 기본적으로 람다항으로 표현이 가능하다. 

예를 들어, 앞서 자연수와 자연수 연산을 람다항으로 이미 앞서 살펴 보았다.
또한 `double`, `quadruple` 등 모두 람다항 형식으로 쉽게 표현할 수 있다.

In [119]:
double x = x + x

* `double` = $\lambda x. \text{plus}\, x\,x$

In [121]:
quadruple x = double (double x)

* `quadruple` = $\lambda x. \text{double}\, (\text{double}\, x)$

#### 하스켈과 튜링-완전성

하스켈이 앞서 소개한 람다 대수와 동일하진 않지만 하스켈도 튜링-완전하다.
이에 대한 자세한 이야기는 여기서 논하지 않는다.

아래 정의가 받아들여 지는 이유를 왼쪽 우선 결합성이 설명한다.

In [114]:
addition x y = x + y
add3 = addition 3